### 1. Import needed packages and clone the dataset

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.datasets import load_files
from keras.utils import np_utils, array_to_img, img_to_array, load_img
from tensorflow.keras.models import Sequential
from keras.layers import Input, Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.callbacks import ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

print('You are using numpy version:', np.__version__)

!git clone https://github.com/Horea94/Fruit-Images-Dataset

In [ ]:
#import sklearn
#print(sklearn.__version__)

Set the path directory to the cloned test and train data.
load_dataset() returns the filenames, integer classes and string classes that are stored in file directory.


1.   names_train is a vector that contains the filepath of all images from the training set
2.   names_test is a vector that contains the filepath of all images from the test set
3.   intclass_train is a vector containing the int class values (1-131) of all images from the training set
4.   intclass_test is a vector containing the int class values (1-131) of all images from the test set
5.   stringclass_train is a vector containing the string label of class of all images from the training set
6.   stringclass_test is a vector containing the string label of class of all images from the test set






In [ ]:
train_dir = 'C:/Users/crysi/Private Dokumente/Papers & wissenschaftl. Arbeiten/Master/Foundations of Machine Learning/First Project/Pycharm/Fruit-Images-Dataset/Training/'
test_dir = 'C:/Users/crysi/Private Dokumente/Papers & wissenschaftl. Arbeiten/Master/Foundations of Machine Learning/First Project/Pycharm/Fruit-Images-Dataset/Test/'

def load_dataset(path):
    data = load_files(path)
    files = np.array(data['filenames'])
    targets = np.array(data['target'])
    target_labels = np.array(data['target_names'])
    return files, targets, target_labels
    
names_train, intclass_train, stringclass_train = load_dataset(train_dir)
names_test, intclass_test, stringclass_test = load_dataset(test_dir)

print('Loading complete!')
print('Training set size : ',  names_train.shape[0])
print('Testing set size : ', names_test.shape[0])
print(stringclass_train)

Show distribution of images to the different classes.

In [ ]:
(intclass, counts) = np.unique(intclass_train, return_counts=True)
plt.bar(intclass, counts)
plt.title('distribution of full training and test data')
full_train_frequencies = np.asarray((intclass, counts)).T

(intclass, counts) = np.unique(intclass_test, return_counts=True)
plt.bar(intclass, counts)
full_test_frequencies = np.asarray((intclass, counts)).T

Show example file paths and their class label

In [ ]:
i = 0
while i < 3:
  print('Path to image : ', names_train[i])
  print('Class label of image: ', intclass_train[i])
  i+=1

In [ ]:
n_classes = len(np.unique(intclass_test))
print('Training set size :',  names_train.shape[0])
print('Testing set size :', names_test.shape[0])
print('Number of unique classes:', n_classes)

Categorized matrix (0 or 1) for class of each image.


1.   train_hot_class is a list containing (length of train images) arrays containing a list each (131 elements) where the i th element is 1, when the image is classified as intclass i (otherwise 0)
2.   test_hot_class is a list containing (length of test images) arrays containing a list each (131 elements) where the i th element is 1, when the image is classified as intclass i (otherwise 0)



In [ ]:
train_hot_class = np_utils.to_categorical(intclass_train, n_classes)
test_hot_class = np_utils.to_categorical(intclass_test, n_classes)
train_hot_class[0]

Change name of image to actual pixel array.
The _images_array are the inputs (100x100 pixels with 3 color channels).


In [ ]:
def convert_image_to_array(files):
    images_as_array=[]
    for file in files:
        # Convert to Numpy Array
        images_as_array.append(img_to_array(load_img(file)))
    return images_as_array

train_images_array = np.array(convert_image_to_array(names_train))
print('Training set shape : ', train_images_array.shape)

test_images_array = np.array(convert_image_to_array(names_test))
print('Test set shape : ', test_images_array.shape)

print('1st training image shape ',train_images_array[0].shape)

Pixel arrays of one image (100x100 pixels, 3 color channels).

In [ ]:
print('1st training image as array',test_images_array[0])

Rescale pixel values from 0-255 range to 0-1.


1.   train_images_array is an array containing the normalized pixel values of the train images.
2.   test_images_array is an array containing the normalized pixel values of the test images.
3.   valid_images_array is an array containing the normalized pixel values of the validation images.



In [ ]:
train_images_array = train_images_array.astype('float32')/255
test_images_array = test_images_array.astype('float32')/255

Split the train datasets into a train dataset and a validation dataset (~75/25).


1.   names_train is a list of names of the first half of the train dataset (actual training dataset)
2.   names_valid is a list of names of the second half of the train dataset (validation dataset)
3.   train_hot_class is the first half of train_hot_class (actual training categorial classes)
4.   valid_hot_class is the second half of train_hot_class (validation categorial classes)



In [ ]:
from sklearn.model_selection import train_test_split
train_images_array, valid_images_array, train_hot_class, valid_hot_class = train_test_split(train_images_array, train_hot_class, test_size = 0.1, random_state=42)

print('Vaildation structure of pixels : ', valid_images_array.shape)
print('Vaildation structure of categorial classes :', valid_hot_class.shape)
print('Train structure of images: ', train_images_array.shape)
print('Train structure of categorial classes : ', train_hot_class.shape)
print('Test structure of images: ', test_images_array.shape)
print('Test structure of categorial classes : ', test_hot_class.shape)





Show 10 images.

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize =(30,5))
for i in range(10):
    ax = fig.add_subplot(2,5,i+1,xticks=[],yticks=[])
    ax.imshow(np.squeeze(train_images_array[i]))
    print(names_train[i])

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize =(30,5))
for i in range(10):
    ax = fig.add_subplot(2,5,i+1,xticks=[],yticks=[])
    ax.imshow(np.squeeze(test_images_array[i]))
    print(names_test[i])

build neural network model

In [ ]:
def build_NeuralNetwork():

    #input layer
    model = Sequential()
    model.add(Input(shape=(100,100,3)))
    model.add(Flatten())

    #hiddenlayer 1
    model.add(Dense(10))
    model.add(BatchNormalization())
    model.add(Activation(activation='relu'))

    #hiddenlayer 2
    model.add(Dense(300)) #900
    model.add(BatchNormalization())
    model.add(Activation(activation='relu'))
    model.add(Dropout(0.5))

    #hiddenlayer 3
    #model.add(Dense(1000)) #900
    #model.add(BatchNormalization())
    #model.add(Activation(activation='relu'))
    #model.add(Dropout(0.5))

    #hiddenlayer 4
    #model.add(Dense(1000)) #900
    #model.add(BatchNormalization())
    #model.add(Activation(activation='relu'))
    #model.add(Dropout(0.5))

    #output layer
    model.add(Dense(131,activation="softmax"))

    model.compile(loss = 'categorical_crossentropy', optimizer = "adam", metrics = ['accuracy'])
    print('Compiled!')

    return model

In [ ]:
network = build_NeuralNetwork()
network.summary()

In [ ]:
checkpointer = ModelCheckpoint(filepath = 'ann.hdf5', verbose = 2, save_best_only = True)
#filename='firstANN.csv'
#history_logger=tf.keras.callbacks.CSVLogger(filename, separator=",", append=True)
history = network.fit(train_images_array, train_hot_class,
    batch_size = 32, #32
    epochs = 25, #15
    validation_data = (test_images_array, test_hot_class),
    verbose = 2,
    callbacks=[checkpointer], shuffle = True)
network.save('ann.h5')

In [ ]:
new_model = tf.keras.models.load_model('ann.h5')
loss, acc = new_model.evaluate(valid_images_array, valid_hot_class, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

In [ ]:
loss, acc = new_model.evaluate(test_images_array, test_hot_class, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))


In [ ]:
model.load_weights('ann.hdf5')
score = model.evaluate(test_images_array, test_hot_class, verbose = 0)
print('\n', 'Test accuracy:', score[1])

In [ ]:
from keras.models import Model
layer_outputs = [layer.output for layer in model.layers]
activation_model = Model(inputs=model.input, outputs=layer_outputs)
activations = activation_model.predict(train_images_array[10].reshape(1,100,100,1))
 
def display_activation(activations, col_size, row_size, act_index): 
    activation = activations[act_index]
    activation_index=0
    fig, ax = plt.subplots(row_size, col_size, figsize=(row_size*2.5,col_size*1.5))
    for row in range(0,row_size):
        for col in range(0,col_size):
            ax[row][col].imshow(activation[0, :, :, activation_index], cmap='gray')
            activation_index += 1

In [ ]:
history.model.weights

In [ ]:
import matplotlib.pyplot as plt
y_pred = model.predict(test_images_array)

fig = plt.figure(figsize=(32, 30))
for i, idx in enumerate(np.random.choice(test_images_array.shape[0], size = 32, replace = False)):
    ax = fig.add_subplot(8, 6, i + 1, xticks = [], yticks = [])
    ax.imshow(np.squeeze(test_images_array[idx]))
    pred_idx = np.argmax(y_pred[idx])
    true_idx = np.argmax(test_hot_class[idx])
    ax.set_title("{} ({})".format(stringclass_test[pred_idx], stringclass_test[true_idx]),
                 color = ("green" if pred_idx == true_idx else "red"))

Model accuracy

In [ ]:
plt.figure(figsize = (6, 5))
plt.plot(history.history['accuracy'], color = 'r')
plt.plot(history.history['val_accuracy'], color = 'b')
plt.title('Model Accuracy', weight = 'bold', fontsize = 16)
plt.ylabel('accuracy', weight = 'bold', fontsize = 14)
plt.xlabel('epoch', weight = 'bold', fontsize = 14)
plt.ylim(0.4, 1)
plt.xticks(weight = 'bold', fontsize = 12)
plt.yticks(weight = 'bold', fontsize = 12)
plt.legend(['train', 'val'], loc = 'lower right', prop = {'size': 14})
plt.grid(color = 'y', linewidth = '0.5')
plt.show()

Show prediction.

In [ ]:
import matplotlib.pyplot as plt 
plt.figure(1)  
   
 # summarize history for accuracy  
   
plt.subplot(211)  
plt.plot(history.history['accuracy'])  
plt.plot(history.history['val_accuracy'])  
plt.title('model accuracy')  
plt.ylabel('accuracy')  
plt.xlabel('epoch')  
plt.legend(['train', 'validation'], loc = 'lower right')
   
 # summarize history for loss  
   
plt.subplot(212)  
plt.plot(history.history['loss'])  
plt.plot(history.history['val_loss'])  
plt.title('model loss')  
plt.ylabel('loss')  
plt.xlabel('epoch')  
plt.legend(['train', 'validation'], loc = 'upper right')
plt.show()